Example notebook for the Young Stellar Objects metric. 
Please contact Loredo Prisidano for more information about the details of the metric. 

The basic premise is to count how many young stellar objects above a SNR cut in g,r, and i bands are available in the coadded images. Dust extinction and distance modulus are taken into account, when searching for stars above a given absolute magnitude. 

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import rubin_sim.maf as maf
from rubin_sim.data import get_baseline


In [2]:
baseline_file = get_baseline()
run_name = os.path.basename(baseline_file).replace('.db','')
outDir = "temp"
resultsDb = maf.db.ResultsDb(out_dir=outDir)

In [4]:
# Set up YSO metric - which counts the number of expected young stellar objects detectable with Rubin

nside = 64
bundleList = []
sql = ""
# Let's plug in the magnitudes for one type
metric = maf.maf_contrib.NYoungStarsMetric()

print("Metric is looking for YSO with the following absolute magnitudes", metric.mags)

slicer = maf.slicers.HealpixSlicer(nside=nside, use_cache=False)

summaryStats = [maf.metrics.SumMetric(), maf.metrics.MaxMetric()]
plot_dict = {"log_scale": True, 'figsize': (8, 6), 
             "color_min": 1, "colorMax":1e5, "extend": "both",
            "cbar_format":'%.0e'}
bundleList.append(
    maf.metricBundles.MetricBundle(
        metric, slicer, sql, plot_dict=plot_dict, summary_metrics=summaryStats, run_name=run_name
    )
)

Metric is looking for YSO with the following absolute magnitudes {'g': 10.32, 'r': 9.28, 'i': 7.97}
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [5]:
bd = maf.metricBundles.make_bundles_dict_from_list(bundleList)
bg = maf.metricBundles.MetricBundleGroup(bd, baseline_file, out_dir=outDir, results_db=resultsDb, verbose=True)
bg.run_all()

Querying table None with no constraint for columns ['fieldDec', 'seeingFwhmGeom', 'rotSkyPos', 'filter', 'fieldRA', 'fiveSigmaDepth'].
Found 2130908 visits
Running:  ['baseline_v3_3_10yrs_young_stars_HEAL']
Read map /Users/lynnej/rubin_sim_data/maps/DustMaps3D/merged_ebv3d_nside64_defaults.fits from disk
Processing slices: 100%|████████████████| 49152/49152 [00:51<00:00, 957.87it/s]
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [6]:
bg.plot_all()

Plotting figures with "" constraint now.


/Users/lynnej/lsst_repos/rubin_sim/rubin_sim/maf/utils/maf_utils.py:143: UserWarning: Optimal bin calculation tried to make 846 bins, returning 200
  warnings.warn(


Plotting complete.


In [7]:
pd.DataFrame([bd[k].summary_values for k in bd], index=list(bd.keys()))

,Sum,Max
baseline_v3_3_10yrs_young_stars_HEAL,4.846930e+06,4188.335665


In [8]:
b = bundleList[0]
print(np.sum(b.metric_values.compressed()), 
      np.median(b.metric_values.compressed()), np.mean(b.metric_values.compressed()), 
      np.max(b.metric_values.compressed()), np.min(b.metric_values.compressed()), 
      np.std(b.metric_values.compressed()))

4846929.655863882 13.96021222185475 144.33117907997982 4188.335665093386 1.4377116280912537e-09 371.4889197232655
